# Projeto 2 - Ciência dos Dados

## > Introdução

### ❑ Objetivos e detalhes da base de dados:

## > Minerando Dados e Características do Dataset

In [1]:
#Importando biblotecas úteis:

%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats

### ❑ Filtros e leitura do Dataset: 

In [4]:
#Iniciando leitura de dados:
raw_data = pd.read_csv('SolarPrediction.csv')
raw_data.head(5)

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00


In [5]:
#Filtrando colunas que serão relevantes para futura análise:
dados = raw_data.loc[:,['Data','Time','Radiation','Pressure','Humidity','WindDirection(Degrees)','Speed']]
dados.head(5)

,Data,Time,Radiation,Pressure,Humidity,WindDirection(Degrees),Speed
0,9/29/2016 12:00:00 AM,23:55:26,1.21,30.46,59,177.39,5.62
1,9/29/2016 12:00:00 AM,23:50:23,1.21,30.46,58,176.78,3.37
2,9/29/2016 12:00:00 AM,23:45:26,1.23,30.46,57,158.75,3.37
3,9/29/2016 12:00:00 AM,23:40:21,1.21,30.46,60,137.71,3.37
4,9/29/2016 12:00:00 AM,23:35:24,1.17,30.46,62,104.95,5.62


### ❑ Descrição de variáveis e análise descritiva:

### ❑ Storytelling geral dos dados:

## > Modelos de Predição 

### ❑ Descrição e aplicação do Modelo 1 (Treinamento):

### ❑ Descrição e aplicação do Modelo 2 (Treinamento):

## > Processo e Estatísticas de Validação dos modelos (Teste)

### ❑ Validação do Modelo 1:

### ❑ Validação do Modelo 2:

## > Conclusão 

## > Referências Bibliográficas 